Задачи по Pandas

V 0.1 24.09.2020

Датасеты:
* addres-book.json
* addres-book.xml
* addres-book-q.xml 

# JSON

* Работаем с файлом addres-book.json

1) Вывести все адреса электронной почты, содржащиеся в адресной книге, имеющей структуру, аналогичную my_ab2.

In [1]:
import json

json_object = json.load(open("./data/addres-book.json"))


json_object

[{'name': 'Faina Lee',
  'email': 'faina@mail.ru',
  'birthday': '22.08.1994',
  'phones': [{'phone': '232-19-55'}, {'phone': '+7 (916) 232-19-55'}]},
 {'name': 'Robert Lee',
  'email': 'robert@mail.ru',
  'birthday': '22.08.1994',
  'phones': [{'phone': '111-19-55'}, {'phone': '+7 (916) 445-19-55'}]}]

In [2]:
[i["email"] for i in json_object]

['faina@mail.ru', 'robert@mail.ru']

2) Вывести телефоны, содржащиеся в адресной книге, имеющей структуру, аналогичную my_ab2.

In [3]:
[i["phones"] for i in json_object]

[[{'phone': '232-19-55'}, {'phone': '+7 (916) 232-19-55'}],
 [{'phone': '111-19-55'}, {'phone': '+7 (916) 445-19-55'}]]

In [3]:
import pandas as pd

json_df = pd.read_json("./data/addres-book.json")

[i for i in json_df["phones"]]

[[{'phone': '232-19-55'}, {'phone': '+7 (916) 232-19-55'}],
 [{'phone': '111-19-55'}, {'phone': '+7 (916) 445-19-55'}]]

3) Изменить структуру данных my_ab2 так, чтобы для телефонов был однозначно известен их тип. Создать пример записи с обновленной структурой данных. Вывести все домашние телефоны, содржащиеся в созданном примере.

In [4]:
import re

df2 = json_df.copy()

for i, plist in enumerate(df2["phones"]):
    for dphon in plist:
        if re.search("(\+7)|(8).*\(...\)", dphon["phone"]):
            dphon["type"]="mobil"
        else:
            dphon["type"]="city"

df2["phones"]

0    [{'phone': '232-19-55', 'type': 'city'}, {'pho...
1    [{'phone': '111-19-55', 'type': 'city'}, {'pho...
Name: phones, dtype: object

3.2) Изменить структуру данных my_ab2 так, чтобы для каждой записи в адресной книге можно было записать список друзей этого человека, присутствующих в этой же адреснойЯфыыв.

In [5]:
df3 = json_df.copy()
df3['friends'] = pd.Series([[] for i in range(len(df3))], index=df3.index)
df3['friends'][0].append("Robert Lee")
df3['friends'][1].append("Faina Lee")
df3

,name,email,birthday,phones,friends
0,Faina Lee,faina@mail.ru,22.08.1994,"[{'phone': '232-19-55', 'type': 'city'}, {'pho...",[Robert Lee]
1,Robert Lee,robert@mail.ru,22.08.1994,"[{'phone': '111-19-55', 'type': 'city'}, {'pho...",[Faina Lee]


4) Написать функцию, которая формирует новую запись в телефонной книге, запрашивая у пользователя поочередно все необходимые поля.

In [6]:
df4 = json_df.copy()
def new_user():
  global df4
  columns = json_df.columns
  name = input("name:")
  email = input("email:")
  birthday = input("birthday:")
  print("phones:")
  city = input("city:")
  mobil = input("mobil:")
  row_list = df4.to_dict(orient="records")
  dict1 = {
    "name": name,
    "email": email,
    "birthday": birthday,
    "phones": [
      {
        "phone": city
      },
      {
       "phone": mobil
      }
    ]}
  row_list.append(dict1)
  df4 = pd.DataFrame(row_list, columns=columns)
new_user()
df4



phones:


,name,email,birthday,phones
0,Faina Lee,faina@mail.ru,22.08.1994,"[{'phone': '232-19-55', 'type': 'city'}, {'pho..."
1,Robert Lee,robert@mail.ru,22.08.1994,"[{'phone': '111-19-55', 'type': 'city'}, {'pho..."
2,,,,"[{'phone': ''}, {'phone': ''}]"


5) Написать функцию, которая ищет по имени в хранящейся в файле телефонной книге запись о человеке и выводит информацию о нем в виде текста, где в каждой строке указывается тип выводимого поля и его содержимое.

In [7]:
def find_user(name):
    columns_len =  max([len(i) for i in json_df.columns])

    for user in json_df[json_df["name"] == name].to_dict(orient="records"):
        for column, value in user.items():
            print(str(column).ljust(columns_len, " "), end=" ")
            if type(value) in [str, int]:
                print(value)
            else:
                print()
                for item in value:
                    print(" "*(columns_len+1) + "phone " + item["phone"])
                print()

find_user(["Faina Lee", "Robert Lee"])

name     Faina Lee
email    faina@mail.ru
birthday 22.08.1994
phones   
         phone 232-19-55
         phone +7 (916) 232-19-55

name     Robert Lee
email    robert@mail.ru
birthday 22.08.1994
phones   
         phone 111-19-55
         phone +7 (916) 445-19-55



6) Получить координаты всех отделений связи г. Пущино, имеющих отдельный почтовый индекс, с использованием API 'http://api.zippopotam.us' . (Индексы использыемые в г. Пущино нужно собрать из открытых источников).

In [9]:
import requests

pushino_indexes = ["142290", "142280"]

urls = [f"https://api.zippopotam.us/ru/{i}" for i in pushino_indexes]

responses = [requests.get(url) for url in urls]

for response in responses:
    for json in response.json()["places"]:
        print(json["longitude"])
        print(json["latitude"])

37.349
54.9454
37.2178
54.8664


# Pickle

1) Сгенирорвать данные, аналогичные структуре ddres-book.json с 1000 записей. 

2) Сохранить результаты з.1 в файл Pickle и в файл JSON. Сравнить объемы получившихся файлов.

3) Сравнить время записи / чтения данных в форматах Pickle и JSON из з.2

4) Преорбазовать структуру данных из ddres-book.json так, чтобы для получения данных о конкретном человеке можно было эффективно использовать возможности модуля shelve.

In [12]:
new_json_df = pd.DataFrame()

new_json_df["name"] = [f"Name #{i}" for i in range(1000000)]
new_json_df["email"] = [f"random{i}@mail" for i in range(1000000)]
new_json_df["birthday"] = [f"01.01.{i+1000}" for i in range(1000000)]
new_json_df["phones"] = [f"123-123-11" for i in range(1000000)]

new_json_df

,name,email,birthday,phones
0,Name #0,random0@mail,01.01.1000,123-123-11
1,Name #1,random1@mail,01.01.1001,123-123-11
2,Name #2,random2@mail,01.01.1002,123-123-11
3,Name #3,random3@mail,01.01.1003,123-123-11
4,Name #4,random4@mail,01.01.1004,123-123-11
...,...,...,...,...
999995,Name #999995,random999995@mail,01.01.1000995,123-123-11
999996,Name #999996,random999996@mail,01.01.1000996,123-123-11
999997,Name #999997,random999997@mail,01.01.1000997,123-123-11
999998,Name #999998,random999998@mail,01.01.1000998,123-123-11


In [13]:
import pickle

pickle.dump(new_json_df, open("./data/pickle_dump", "wb"))

new_json_df.to_json("./data/json_dump.json")

In [14]:
import os

f"Pickle {os.stat('./data/pickle_dump').st_size} байт"

'Pickle 54687127 байт'

In [15]:
f"JSON {os.stat('./data/json_dump.json').st_size} байт"

'JSON 98226384 байт'

In [21]:
import time
import numpy as np

pickle_times = []

for i in range(10):

    time_start = time.time()

    pickle.load(open("./data/pickle_dump", "rb"))

    total_time = time.time() - time_start

    pickle_times.append(total_time)

print(np.mean(pickle_times))

pickle_times_mean = np.mean(pickle_times)

0.33846981525421144


In [22]:
json_times = []

for i in range(100):

    time_start = time.time()

    pd.read_json("./data/addres-book.json")

    total_time = time.time() - time_start

    json_times.append(total_time)

print(np.mean(json_times))

json_times_mean = np.mean(json_times)

print(f"Pickle в среднем быстрее JSON в: {json_times_mean / pickle_times_mean} раз")

# Pickle должен быть быстрее JSON в среднем в 7 раз, python почему-то очень долго вызывает pickle.load, хотя раньше всё было быстро..

0.002956600189208984
Pickle в среднем быстрее JSON в: 0.008735196038052603 раз


4) Преорбазовать структуру данных из ddres-book.json так, чтобы для получения данных о конкретном человеке можно было эффективно использовать возможности модуля shelve.

In [23]:
json_file = pd.read_json("./data/addres-book.json")

json_file["phone_home"] = [[z["phone"] for z in i]for i in json_df["phones"]][0]
json_file["mobile_phone"] = [[z["phone"] for z in i]for i in json_df["phones"]][1]

json_file = json_file.drop("phones", axis=1)

json_file

,name,email,birthday,phone_home,mobile_phone
0,Faina Lee,faina@mail.ru,22.08.1994,232-19-55,111-19-55
1,Robert Lee,robert@mail.ru,22.08.1994,+7 (916) 232-19-55,+7 (916) 445-19-55


In [28]:
import shelve

shelve_file = shelve.open("data/shelve_file.db")

for name in json_file["name"]:
    shelve_file[name] = json_file[json_file["name"] == name]

print(shelve_file["Robert Lee"])

shelve_file.close()

         name           email    birthday          phone_home  \
1  Robert Lee  robert@mail.ru  22.08.1994  +7 (916) 232-19-55   

         mobile_phone  
1  +7 (916) 445-19-55  


In [25]:
shelve_file = shelve.open("data/shelve_file.db")
shelve_file["Robert Lee"]


,name,email,birthday,phone_home,mobile_phone
1,Robert Lee,robert@mail.ru,22.08.1994,+7 (916) 232-19-55,+7 (916) 445-19-55


In [29]:
with shelve.open("data/shelve_file.db") as shelve_file:
    for item in shelve_file.items():
        print(item)

('Faina Lee',         name          email    birthday phone_home mobile_phone
0  Faina Lee  faina@mail.ru  22.08.1994  232-19-55    111-19-55)
('Robert Lee',          name           email    birthday          phone_home  \
1  Robert Lee  robert@mail.ru  22.08.1994  +7 (916) 232-19-55   

         mobile_phone  
1  +7 (916) 445-19-55  )


# XML

1) По данным из файла addres-book-q.xml сформировать список словарей с телефоноами каждого из людей. Вид результата: [{ФИО: ['телефон1', 'телефон2']}] и вывести его на экран и сохранить в json файле.

2) По данным из файла addres-book-q.xml сформировать список (в виде списка в Python) адресов электронной почты всех женщин и вывести его на экран

3) По данным из файла addres-book-q.xml сформировать список (в виде списка в Python) всех участников с рабочими телефонами и вывести его на экран

4) По данным из файла addres-book-q.xml сформировать словарь вида {название страны: количество участников} и вывести его на экран


In [32]:
# 1) По данным из файла addres-book-q.xml сформировать список словарей с телефоноами каждого из людей. Вид результата: [{ФИО: ['телефон1', 'телефон2']}] и вывести его на экран и сохранить в json файле.

import xml.etree.ElementTree as ET
import json

adb = []

tree = ET.parse('data/addres-book-q.xml')
root = tree.getroot()
print(root.tag)
for country in root:
    print("\t",country.tag, country.attrib)
    for address in country:
        print("\t    ",address.tag, address.attrib)
        phones = address.find("phones")
        for phone in phones:
            print("\t\t",phone.tag, phone.attrib, phone.text)
        adb.append({address.find("name").text:[phone.text for phone in phones]})

with open('data.json', 'w') as f:
    json.dump(adb, f)

display(adb)


address_book
	 country {'name': 'algeria'}
	     address {'id': '1'}
		 phone {'type': 'work'} + (213) 6150 4015
		 phone {'type': 'personal'} + (213) 2173 5247
	 country {'name': 'angola'}
	     address {'id': '2'}
		 phone {'type': 'work'} + (244-2) 325 023
		 phone {'type': 'personal'} + (244-2) 325 023
	     address {'id': '3'}
		 phone {'type': 'personal'} + (244) 4232 2836
	     address {'id': '4'}
		 phone {'type': 'work'} + (244-2) 325 023
		 phone {'type': 'personal'} + (244-2) 325 023
	 country {'name': 'argentina'}
	     address {'id': '5'}
		 phone {'type': 'work'} + (54-11) 4784 1159
	 country {'name': 'australia'}
	     address {'id': '6'}
		 phone {'type': 'work'} + (61-2) 6274 9500
		 phone {'type': 'personal'} + (61-2) 6274 9513
	     address {'id': '7'}
		 phone {'type': 'work'} + (61-3) 9807 4702


[{'Aicha Barki': ['+ (213) 6150 4015', '+ (213) 2173 5247']},
 {'Francisco Domingos': ['+ (244-2) 325 023', '+ (244-2) 325 023']},
 {'Maria Luisa': ['+ (244) 4232 2836']},
 {'Abraao Chanda': ['+ (244-2) 325 023', '+ (244-2) 325 023']},
 {'Beatriz Busaniche': ['+ (54-11) 4784 1159']},
 {'Francesca Beddie': ['+ (61-2) 6274 9500', '+ (61-2) 6274 9513']},
 {'Graham John Smith': ['+ (61-3) 9807 4702']}]

In [34]:
# 2) По данным из файла addres-book-q.xml сформировать список (в виде списка в Python) адресов электронной почты всех женщин и вывести его на экран

mails = []

tree = ET.parse('data/addres-book-q.xml')
root = tree.getroot()
for country in root:
    for address in country:
        gender = address.find("gender").text
        if gender == "f":
            email = address.find("email")
            mails.append(email.text)
mails

['luisagrilo@ebonet.net', 'f.beddie@ala.asn.au']

In [36]:
# 3) По данным из файла addres-book-q.xml сформировать список (в виде списка в Python) всех участников с рабочими телефонами и вывести его на экран

idList = []

tree = ET.parse('data/addres-book-q.xml')
root = tree.getroot()
for country in root:
    for address in country:
        phones = address.find("phones")
        if "work" in [phone.get("type") for phone in phones]:
            idList.append(address.get("id"))

for id in idList:
    print(root.find(f".//*[@id='{id}']").find("name").text)

Aicha Barki
Francisco Domingos
Abraao Chanda
Beatriz Busaniche
Francesca Beddie
Graham John Smith


In [39]:
# 4) По данным из файла addres-book-q.xml сформировать словарь вида {название страны: количество участников} и вывести его на экран

countryUsers = dict()

tree = ET.parse('data/addres-book-q.xml')
root = tree.getroot()
for country in root:
    countryUsers[country.get("name")] = len(country)

countryUsers

{'algeria': 1, 'angola': 3, 'argentina': 1, 'australia': 2}

In [41]:
# 5) Превратить информацию из addres-book.xml в json файл с содержимым вида: [{'birthday': '11.07.1984', 'email': 'bruce@gmail.com', 'name': 'Bruce Lee'}, {'birthday': '22.03.1985', 'email': 'alee@yandex.ru', 'name': 'Alice Lee'}]

import pandas as pd

df = pd.read_xml("data/addres-book.xml", )
df.drop(columns=["phones","work"], inplace=True)
display(dictdf:=df.to_dict(orient="records"))

with open('data_2.json', 'w') as f:
    json.dump(dictdf, f)

[{'id': 1,
  'name': 'Bruce Lee',
  'email': 'bruce@gmail.com',
  'birthday': '11.07.1984'},
 {'id': 2,
  'name': 'Alice Lee',
  'email': 'alee@yandex.ru',
  'birthday': '22.03.1985'}]

Здачи на работу с данными ЕЦБ:

In [15]:
import requests
from bs4 import BeautifulSoup

In [16]:
# Create a variable with the URL to this tutorial
url = 'http://sdw.ecb.europa.eu/quickviewexport.do?SERIES_KEY=320.MNA.Q.Y.I8.W0.S1.S1._Z.LPR_PS._Z._T._Z.EUR.LR.GY&start=01-01-2001&end=01-01-2011&type=sdmx'

# Scrape the HTML at the url
r = requests.get(url)

# Turn the HTML into a Beautiful Soup object
s = BeautifulSoup(r.text, 'xml')

In [17]:
s.MessageGroup.Header

<Header>
<ID>IREF000001</ID>
<Test>false</Test>
<Prepared>2022-10-10T13:03:35Z</Prepared>
<Sender id="4F0">
<Name xml:lang="en">European Central Bank</Name>
<Contact>
<Department xml:lang="en">DG Statistics</Department>
<URI>mailto:statistics@ecb.europa.eu</URI></Contact>
</Sender>
<Extracted>2022-10-10T13:03:35Z</Extracted>
</Header>

In [18]:
s.MessageGroup.Header.ID

<ID>IREF000001</ID>

In [19]:
s.MessageGroup.Header.ID.text

'IREF000001'

In [20]:
obs_all = s.findAll('Obs')

In [21]:
len(obs_all)

41

In [22]:
for obs in obs_all:
    print('When: {}, value {}'.format(obs['TIME_PERIOD'], obs['OBS_VALUE']))

When: 2001-Q1, value 1.4
When: 2001-Q2, value 1.0
When: 2001-Q3, value 0.9
When: 2001-Q4, value 0.5
When: 2002-Q1, value -0.4
When: 2002-Q2, value 0.0
When: 2002-Q3, value 0.3
When: 2002-Q4, value 0.6
When: 2003-Q1, value 0.5
When: 2003-Q2, value -0.0
When: 2003-Q3, value 0.1
When: 2003-Q4, value 0.5
When: 2004-Q1, value 1.2
When: 2004-Q2, value 1.7
When: 2004-Q3, value 1.3
When: 2004-Q4, value 0.9
When: 2005-Q1, value 0.6
When: 2005-Q2, value 0.5
When: 2005-Q3, value 1.0
When: 2005-Q4, value 1.0
When: 2006-Q1, value 1.4
When: 2006-Q2, value 1.7
When: 2006-Q3, value 1.3
When: 2006-Q4, value 1.9
When: 2007-Q1, value 1.5
When: 2007-Q2, value 1.2
When: 2007-Q3, value 1.1
When: 2007-Q4, value 0.5
When: 2008-Q1, value 0.5
When: 2008-Q2, value -0.0
When: 2008-Q3, value -0.5
When: 2008-Q4, value -2.1
When: 2009-Q1, value -4.4
When: 2009-Q2, value -3.5
When: 2009-Q3, value -2.3
When: 2009-Q4, value -0.3
When: 2010-Q1, value 2.6
When: 2010-Q2, value 2.9
When: 2010-Q3, value 2.5
When: 2010-Q4, v

## Сбор данных с html страниц

In [1]:
import urllib
import urllib.parse
import urllib.request

In [2]:
from bs4 import BeautifulSoup

url_wikipedia = 'https://ru.wikipedia.org'
url_es = 'https://ru.wikipedia.org/wiki/Категория:Электростанции_по_алфавиту'
url_es_q  = urllib.parse.quote(url_es, safe='/:')
print(url_es_q)

https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%AD%D0%BB%D0%B5%D0%BA%D1%82%D1%80%D0%BE%D1%81%D1%82%D0%B0%D0%BD%D1%86%D0%B8%D0%B8_%D0%BF%D0%BE_%D0%B0%D0%BB%D1%84%D0%B0%D0%B2%D0%B8%D1%82%D1%83


In [7]:
es_bs = BeautifulSoup(urllib.request.urlopen(url_es_q), 'lxml')

In [8]:
all_es = []
for i in range(30):
    # load links:
    bef = len(all_es)
    for links in es_bs.find_all('div', class_='mw-category-group'):
        for l in links.find_all('a'):
            all_es.append((l['title'], url_wikipedia+l['href']))

In [27]:
all_es

[('Абаданская ГЭС',
  'https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D0%B0%D0%B4%D0%B0%D0%BD%D1%81%D0%BA%D0%B0%D1%8F_%D0%93%D0%AD%D0%A1'),
 ('Абаканская ТЭЦ',
  'https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D0%B0%D0%BA%D0%B0%D0%BD%D1%81%D0%BA%D0%B0%D1%8F_%D0%A2%D0%AD%D0%A6'),
 ('Автовская ТЭЦ',
  'https://ru.wikipedia.org/wiki/%D0%90%D0%B2%D1%82%D0%BE%D0%B2%D1%81%D0%BA%D0%B0%D1%8F_%D0%A2%D0%AD%D0%A6'),
 ('Автозаводская ТЭЦ',
  'https://ru.wikipedia.org/wiki/%D0%90%D0%B2%D1%82%D0%BE%D0%B7%D0%B0%D0%B2%D0%BE%D0%B4%D1%81%D0%BA%D0%B0%D1%8F_%D0%A2%D0%AD%D0%A6'),
 ('Аджарис-Цкальская ГЭС',
  'https://ru.wikipedia.org/wiki/%D0%90%D0%B4%D0%B6%D0%B0%D1%80%D0%B8%D1%81-%D0%A6%D0%BA%D0%B0%D0%BB%D1%8C%D1%81%D0%BA%D0%B0%D1%8F_%D0%93%D0%AD%D0%A1'),
 ('Адлерская ТЭС',
  'https://ru.wikipedia.org/wiki/%D0%90%D0%B4%D0%BB%D0%B5%D1%80%D1%81%D0%BA%D0%B0%D1%8F_%D0%A2%D0%AD%D0%A1'),
 ('Адыгейская ВЭС',
  'https://ru.wikipedia.org/wiki/%D0%90%D0%B4%D1%8B%D0%B3%D0%B5%D0%B9%D1%81%D0%BA%D0%B0%D1%8F_%D0%92%D0%AD%D0%A1'

In [28]:
all_es_prop = []

In [29]:
for i, (n, lnk) in enumerate(all_es[:10]):
    es = BeautifulSoup(urllib.request.urlopen(lnk), 'lxml')
    coord = es.find_all('a', class_='mw-kartographer-maplink')
    if len(coord) > 0:
        lat, lon = coord[0]['data-lat'], coord[0]['data-lon']
    else:
        print('{}: {}, COORDINATES IS NOT FOUND! link: {}'.format(i, n, lnk))        
        continue
                          
    es_pr = {}
    table = es.find('table', class_="infobox")
    if table is not None:
        for row in table.find_all('tr'):
            th = row.find_all('th')
            if th and len(th):
    #             print(th[0].text.strip(), end=' || ')
                td = row.find_all('td')
                if td and len(td):
                    es_pr[th[0].text.strip()] = td[0].text.strip()
        #             print(td[0].text.strip())            
        all_es_prop.append((n, lnk, lat, lon, es_pr))
        print('{}: {}, lat: {}, lon: {}, cont: {}'.format(i, n, lat, lon, es_pr.get('Страна', '--')))
    else:
        print('{}: {}, TABLE IS NOT FOUND! link: {}'.format(i, n, lnk)) 

0: Абаданская ГЭС, lat: 38.06, lon: 58.16194, cont: Туркмения
1: Абаканская ТЭЦ, lat: 53.71944, lon: 91.32194, cont: Россия
2: Автовская ТЭЦ, lat: 59.8700861, lon: 30.2868833, cont: Россия
3: Автозаводская ТЭЦ, lat: 56.24333, lon: 43.8925, cont: Россия
4: Аджарис-Цкальская ГЭС, lat: 41.570306, lon: 41.861472, cont: Грузия
5: Адлерская ТЭС, lat: 43.420083, lon: 39.94875, cont: Россия
6: Адыгейская ВЭС, lat: 44.935, lon: 40.145, cont: Россия
7: Азербайджанская ТЭС, lat: 40.78, lon: 46.99, cont: Азербайджан
8: Ак-Кавакская ГЭС-1, lat: 41.447083, lon: 69.558167, cont: Узбекистан
9: Академик Ломоносов, lat: 69.70975, lon: 170.309889, cont: Россия


In [30]:
all_es_prop

[('Абаданская ГЭС',
  'https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D0%B0%D0%B4%D0%B0%D0%BD%D1%81%D0%BA%D0%B0%D1%8F_%D0%93%D0%AD%D0%A1',
  '38.06',
  '58.16194',
  {'Страна': 'Туркмения',
   'Местоположение': 'Ашхабад, Абаданский этрап',
   'Собственник': 'Туркменэнерго',
   'Статус': 'Действующая',
   'Ввод в эксплуатацию': '1957',
   'Электрическая мощность, МВт': '371',
   'Основное топливо': 'природный газ',
   'Количество и марка генераторов': '2 (General Electric, Çalyk Enerji)'}),
 ('Абаканская ТЭЦ',
  'https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D0%B0%D0%BA%D0%B0%D0%BD%D1%81%D0%BA%D0%B0%D1%8F_%D0%A2%D0%AD%D0%A6',
  '53.71944',
  '91.32194',
  {'Страна': 'Россия',
   'Местоположение': 'город Абакан, Хакасия',
   'Статус': 'действующая',
   'Ввод в эксплуатацию': '1982',
   'Электрическая мощность, МВт': '406 МВт',
   'Тепловая мощность': '625 Гкал/ч',
   'Основное топливо': 'бурый уголь Ирша-Бородинского месторождения Канско-Ачинского угольного\xa0бассейна',
   'Резервное топливо':

Задание: собрать данные о городах России по категори: https://ru.wikipedia.org/wiki/Категория:Города_по_субъектам_Российской_Федерации


In [3]:
url_wikipedia = 'https://ru.wikipedia.org'
url_es = 'https://ru.wikipedia.org/wiki/Категория:Города_по_субъектам_Российской_Федерации'
url_es_q  = urllib.parse.quote(url_es, safe='/:')
print(url_es_q)

https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0_%D0%BF%D0%BE_%D1%81%D1%83%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D0%B0%D0%BC_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B9%D1%81%D0%BA%D0%BE%D0%B9_%D0%A4%D0%B5%D0%B4%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D0%B8


In [4]:
es_bs = BeautifulSoup(urllib.request.urlopen(url_es_q), 'lxml')

In [5]:
all_es = []
for i in range(30):
    # load links:
    bef = len(all_es)
    for links in es_bs.find_all('div', class_='mw-category-group'):
        for l in links.find_all('a'):
            all_es.append((l['title'], url_wikipedia+l['href']))

In [6]:
all_es

[('Категория:Категории городов по субъектам Российской Федерации',
  'https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D0%B8_%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D0%BE%D0%B2_%D0%BF%D0%BE_%D1%81%D1%83%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D0%B0%D0%BC_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B9%D1%81%D0%BA%D0%BE%D0%B9_%D0%A4%D0%B5%D0%B4%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D0%B8'),
 ('Категория:Города Крыма',
  'https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0_%D0%9A%D1%80%D1%8B%D0%BC%D0%B0'),
 ('Категория:Категории по городам субъектов Российской Федерации',
  'https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D0%B8_%D0%BF%D0%BE_%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D0%B0%D0%BC_%D1%81%D1%83%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D0%BE%D0%B2_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B9%D1%81%D0%BA%D0%

In [100]:
import pandas as pd
all_es_prop = []

for i, (n, lnk) in enumerate(all_es[:9]):
    es = BeautifulSoup(urllib.request.urlopen(lnk), 'lxml')
    es_pr = {}
    city_info = {}

    all_cities_in_group = []
    mw_pages = es.find("div", {"id": "mw-pages"})
    group_cities_links = []
    saved_th = []
    cities_rows = []

    cities = []

    df_parsed = pd.DataFrame()

    if mw_pages is not None:
        group_cities = mw_pages.find_all("a")
        for each_a in group_cities:
            if "Городские" in each_a["title"]:
                group_cities_links.append(f"https://ru.wikipedia.org{each_a['href']}")
                all_cities_in_group.append(each_a["title"])
                #print(each_a["href"], each_a["title"])
        #print(group_cities)
        #all_cities_in_group.append(group_cities["title"])
        #print(f"https://ru.wikipedia.org{group_cities['href']}")
        #group_cities_links.append(f"https://ru.wikipedia.org{group_cities['href']}")

    for link in group_cities_links:
        cs = BeautifulSoup(urllib.request.urlopen(link), 'lxml')
        cities_table = cs.find("table", class_="standard")
        #print(link)
        if cities_table is not None:
            for row in cities_table.find_all("tr"):
                th = row.find_all("th")
                #print("123")
                #print(th.text)
                if th:
                    #pass
                    #print(f"Th length: {len(th)}")
                    for i in range(len(th)):

                        #print(th[i].text.strip(), end=' || ')
                        saved_th.append(th[i].text.strip())
                        #df_parsed[th[i].text.strip()] = 0
                td = row.find_all('td')
                #print(td.text)
                if td:
                    #print("123")
                    for i in range(len(td)):
                        a_img = td[i].find_all("a", class_="image")

                        if a_img:
                            for each_img in a_img:
                                img_src = each_img.find_all("img")
                                if img_src:
                                    for each_src in img_src:

                                        image_link = each_src["src"]

                                        # print(f"Th[i]: {i}")
                                        # print(f"Th now: {saved_th}")
                                        # print(f"Th length now: {len(saved_th)}")
                                        # print(f"Image link in: {saved_th[i]}")

                        #city_info[saved_th[i]] = td[i].text.strip()
                        #es_pr[td[i].text.strip() if saved_th[i] == "№" else len(es_pr)] = {saved_th[i]: td[i].text.strip()}

                        es_pr[saved_th[i]] = td[i].text.strip()
                        if i == len(td)- 1:
                            #print(es_pr)
                            cities.append(es_pr)
                            #print(pd.json_normalize(es_pr))

                            es_pr = {}
                        #df_parsed[saved_th[i]] = td[i].text.strip()
                        #print(td[i].text.strip(), "----", saved_th[i])
                    #es_pr[td[i].text.strip()] = city_info
                    #print(cities)
                    #print("123")
    if len(cities) > 0:
        print(cities)
        #print(es_pr)
        all_es_prop.append((n, {"Города": cities}))
        df_parsed[n] = cities
        #print('{}: {}, Название города: {}'.format(i, n, city_name))


[{'№': '1', 'название': 'Алупка', 'район[5] / город   республиканского значения[6]': 'Ялта', 'население(чел.)': '↗9063[4]', 'основание / первое упоминание': '960', 'статус города': '1938', 'герб': ''}, {'№': '2', 'название': 'Алушта', 'район[5] / город   республиканского значения[6]': 'Алушта', 'население(чел.)': '↗31\xa0364[4]', 'основание / первое упоминание': 'VI в.', 'статус города': '1902', 'герб': ''}, {'№': '3', 'название': 'Армянск', 'район[5] / город   республиканского значения[6]': 'Армянск', 'население(чел.)': '↘20\xa0692[4]', 'основание / первое упоминание': '1736', 'статус города': '1993', 'герб': ''}, {'№': '4', 'название': 'Бахчисарай', 'район[5] / город   республиканского значения[6]': 'Бахчисарайский район', 'население(чел.)': '↗28\xa0609[4]', 'основание / первое упоминание': '1389', 'статус города': '', 'герб': ''}, {'№': '5', 'название': 'Белогорск', 'район[5] / город   республиканского значения[6]': 'Белогорский район', 'население(чел.)': '↗17\xa0445[4]', 'основание

In [101]:
df_parsed

,Категория:Города Астраханской области
0,"{'№': '1', 'название': 'Астрахань', 'район / З..."
1,"{'№': '2', 'название': 'Ахтубинск', 'район / З..."
2,"{'№': '3', 'название': 'Знаменск', 'район / ЗА..."
3,"{'№': '4', 'название': 'Камызяк', 'район / ЗАТ..."
4,"{'№': '5', 'название': 'Нариманов', 'район / З..."
5,"{'№': '6', 'название': 'Харабали', 'район / ЗА..."


In [102]:
all_es_prop

[('Категория:Города Крыма',
  {'Города': [{'№': '1',
     'название': 'Алупка',
     'район[5] / город   республиканского значения[6]': 'Ялта',
     'население(чел.)': '↗9063[4]',
     'основание / первое упоминание': '960',
     'статус города': '1938',
     'герб': ''},
    {'№': '2',
     'название': 'Алушта',
     'район[5] / город   республиканского значения[6]': 'Алушта',
     'население(чел.)': '↗31\xa0364[4]',
     'основание / первое упоминание': 'VI в.',
     'статус города': '1902',
     'герб': ''},
    {'№': '3',
     'название': 'Армянск',
     'район[5] / город   республиканского значения[6]': 'Армянск',
     'население(чел.)': '↘20\xa0692[4]',
     'основание / первое упоминание': '1736',
     'статус города': '1993',
     'герб': ''},
    {'№': '4',
     'название': 'Бахчисарай',
     'район[5] / город   республиканского значения[6]': 'Бахчисарайский район',
     'население(чел.)': '↗28\xa0609[4]',
     'основание / первое упоминание': '1389',
     'статус города': '',

In [108]:
print(all_es_prop[1])

('Категория:Города Адыгеи', {'Города': [{'№': '1', 'название': 'Адыгейск', 'город республиканского значения (городской округ)': 'Адыгейск', 'население(чел.)': '↗12\xa0910[3]', 'основан': '1969', 'статусгорода': '1976', 'герб': '', 'прежнее название': 'до 1976 — Адыгейский\nдо 1990 — Теучежск'}, {'№': '2', 'название': 'Майкоп', 'город республиканского значения (городской округ)': 'Майкоп', 'население(чел.)': '↗143\xa0385[4]', 'основан': '1857', 'статусгорода': '1870', 'герб': '', 'прежнее название': ''}]})


In [114]:
pd.json_normalize(all_es_prop[0][1]["Города"])

,№,название,район[5] / город республиканского значения[6],население(чел.),основание / первое упоминание,статус города,герб
0,1,Алупка,Ялта,↗9063[4],960,1938,
1,2,Алушта,Алушта,↗31 364[4],VI в.,1902,
2,3,Армянск,Армянск,↘20 692[4],1736,1993,
3,4,Бахчисарай,Бахчисарайский район,↗28 609[4],1389,,
4,5,Белогорск,Белогорский район,↗17 445[4],XIII в.,,
5,6,Джанкой,Джанкой,↘37 014[4],1855,1917,
6,7,Евпатория,Евпатория,↘107 877[4],497 до н. э.,,
7,8,Керчь,Керчь,↗154 621[4],VII в. до н. э.,,
8,9,Красноперекопск,Красноперекопск,↗25 569[4],1932,1966,
9,10,Саки,Саки,↘24 285[4],,1952,


In [73]:

pd.json_normalize(all_es_prop[1])

,Города
0,NaN
1,"[{'1': '№', 'Адыгейск': 'город республиканског..."


In [ ]:
parsed_df = pd.DataFrame()

for i in range(len(all_es_prop)):


In [ ]:
import